# Set Up

## Check GPU

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib 
tf.__version__

In [ ]:
tf.config.list_physical_devices('GPU')

## Define Helper Functions

In [ ]:
def plot_loss_acc(history):
    '''
    Plots the training and validation accuracy and loss curves for fitted keras models
    In: The history object of a trained keras model
    Out: 2 Plots (Accuracy & Loss)
    '''
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,5))  
    fig.suptitle('Model Performance')

    # summarize history for accuracy
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_title('Training and Validation Accuracy')
    ax1.set(xlabel='Epoch', ylabel='Accuracy')    
    ax1.legend(['Training Accuracy', 'Validation Accuracy'], loc='best')
    
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('Training and Validation Loss')
    ax2.set(xlabel='Epoch', ylabel='Loss') 
    ax2.legend(['Training Loss', 'Validation Loss'], loc='best')
    
    return fig

def plot_conf_mat(model, load=False, load_loc=None, class_lst = 'Angry Disgust Fear Happy Sad Surprise Neutral', out='model_weights/finalconf.png'):
    import seaborn as sns
    # 0=angry, 1=disgust,2=fear,3=happy, 4=sad, 5=surprise, 6=neutral
    classes = class_lst.split(' ')
    y_true=[np.argmax(x) for x in val_data[1]]

    if load:
        model.load_weights(load_loc)
        mod_name = load_loc.split('-')[0]
    else:
        mod_name=out


    y_pred=np.argmax(model.predict(val_data),axis=1) 
    con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()

    con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

    con_mat_df = pd.DataFrame(con_mat_norm,
                         index = classes, 
                         columns = classes)

    figure = plt.figure(figsize=(8, 8))
    ax = sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.BuGn)
    ax.set_ylim(len(classes),0)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.gcf().subplots_adjust(bottom=0.15, left=0.15)
    plt.savefig(mod_name+'_performance.png')
    return figure

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 4.0 #was 2.0
    return x

def split_data(x, y, validation_split=.2):
    num_samples = len(x)
    num_train_samples = int((1 - validation_split)*num_samples)
    train_x = x[:num_train_samples]
    train_y = y[:num_train_samples]
    val_x = x[num_train_samples:]
    val_y = y[num_train_samples:]
    train_data = (train_x, train_y)
    val_data = (val_x, val_y)
    return train_data, val_data

# Build Models

In [ ]:
import pandas as pd
import numpy as np
import numpy
import matplotlib.pyplot as plt
import cv2

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense,\
Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, SeparableConv2D,\
GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# from tensorflow.keras.optimizers import Adam


%matplotlib inline

## Parameter Set Up

In [ ]:
# dimensions of our images.
image_size=(48, 48)
image_width, image_height = 48, 48
num_classes = 7

# train_data_dir = 'FERPlus-master/data/FER2013Train'
# validation_data_dir = 'FERPlus-master/data/FER2013Valid'
nb_train_samples = 28559
nb_validation_samples = 3580 
num_epochs = 150
batch_size = 64

dataset_name = 'emotion'
patience = 50

if K.image_data_format() == 'channels_first':
    input_shape = (1, image_width, image_height)
else:
    input_shape = (image_width, image_height, 1)
    
img_input = Input(input_shape)

## Import Data

In [ ]:
# FER2013 --

raw_df = pd.read_csv('../data/fer2013_and_plus.csv')
pixels = raw_df['pixels'].tolist()
faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(image_width, image_height)
    face = cv2.resize(face.astype('uint8'), image_size)
    faces.append(face.astype('float32'))
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)
emotions = pd.get_dummies(raw_df[dataset_name]).as_matrix()

faces = preprocess_input(faces)
num_samples, num_classes = emotions.shape
train_data, val_data = split_data(faces, emotions, validation_split=0.2)
train_faces, train_emotions = train_data

In [ ]:
train_data[1].shape

## Set Up Data Generators

In [ ]:
data_generator = ImageDataGenerator(
                        #rescale=1. / 255,
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

test_datagen = ImageDataGenerator()#rescale=1. / 255)



## Set Up Model Architecture

### MODEL: big XCEPTION

In [ ]:
#big XCEPTION
x = Conv2D(32, (3, 3), strides=(2, 2), use_bias=False)(img_input)
x = BatchNormalization(name='block1_conv1_bn')(x)
x = Activation('relu', name='block1_conv1_act')(x)
x = Conv2D(64, (3, 3), use_bias=False)(x)
x = BatchNormalization(name='block1_conv2_bn')(x)
x = Activation('relu', name='block1_conv2_act')(x)

residual = Conv2D(128, (1, 1), strides=(2, 2),
                  padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
x = BatchNormalization(name='block2_sepconv1_bn')(x)
x = Activation('relu', name='block2_sepconv2_act')(x)
x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
x = BatchNormalization(name='block2_sepconv2_bn')(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

residual = Conv2D(256, (1, 1), strides=(2, 2),
                  padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = Activation('relu', name='block3_sepconv1_act')(x)
x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
x = BatchNormalization(name='block3_sepconv1_bn')(x)
x = Activation('relu', name='block3_sepconv2_act')(x)
x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
x = BatchNormalization(name='block3_sepconv2_bn')(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
x = Conv2D(num_classes, (3, 3),
           # kernel_regularizer=regularization,
           padding='same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax', name='predictions')(x)

model = Model(img_input, output)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# big XCEPTION callbacks
log_file_path = '../models/logs/' + dataset_name + '_training.log'
csv_logger = CSVLogger(log_file_path, append=False)

early_stop = EarlyStopping('val_loss', patience=patience)

reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)

trained_models_path = '../models/weights/emotion/' + dataset_name + '_big_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, monitor='val_accuracy', verbose=1, save_best_only=True)

callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

In [ ]:
history_bigXception = model.fit_generator(data_generator.flow(train_faces, train_emotions,batch_size),
                    steps_per_epoch=len(train_faces) / batch_size,
                    epochs=num_epochs, verbose=1, callbacks=callbacks,
                    validation_data=val_data)

In [ ]:
fig = plot_loss_acc(history_bigXception)
#fig.savefig('model_weights/big XCEPTION curves')
fig.show()

In [ ]:
from glob import glob
file_lst = sorted(glob('model_weights/emotion_mini_XCEPTION*.hdf5'))
best_acc=0
for i,filename in enumerate(file_lst):
    if int(filename.split('.')[-2])>best_acc:
        best_acc = int(filename.split('.')[-2])
        best_idx=i
best_loc = file_lst[best_idx]
best_loc

In [ ]:
# fig = plot_conf_mat(model, load=True, load_loc='./model_weights/emotion_big_XCEPTION.28-0.64.hdf5', 
#               class_lst = 'Angry Disgust Fear Happy Sad Surprise Neutral', out='model_weights/finalconf.png')
# fig.show()

### MODEL: mini XCEPTION

In [ ]:
#mini_XCEPTION
from tensorflow.keras.regularizers import l2
l2_regularization=0.06

regularization = l2(l2_regularization)

# base
img_input = Input(input_shape)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
           use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
           use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# module 1
residual = Conv2D(16, (1, 1), strides=(2, 2),
                  padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(16, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

# module 2
residual = Conv2D(32, (1, 1), strides=(2, 2),
                  padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(32, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

# module 3
residual = Conv2D(64, (1, 1), strides=(2, 2),
                  padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(64, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

# module 4
residual = Conv2D(128, (1, 1), strides=(2, 2),
                  padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(128, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

x = Conv2D(num_classes, (3, 3),
           # kernel_regularizer=regularization,
           padding='same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax', name='predictions')(x)

model = Model(img_input, output)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# mini XCEPTION callbacks
log_file_path = '../models/logs/' + dataset_name + '_training_mini_xception.log'
csv_logger = CSVLogger(log_file_path, append=False)

early_stop = EarlyStopping('val_loss', patience=patience)

reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)

trained_models_path = '../models/weights/emotion/' + dataset_name + '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, monitor='val_loss', verbose=1, save_best_only=True)

callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

In [ ]:
history_miniXception = model.fit_generator(data_generator.flow(train_faces, train_emotions,batch_size),
                    steps_per_epoch=len(train_faces) / batch_size,
                    epochs=num_epochs, verbose=1,
                    validation_data=val_data, callbacks=callbacks)

In [ ]:
model.save_weights('final_weights_emotion.hdf5')

In [ ]:
fig = plot_loss_acc(history_miniXception)
fig.savefig('../models/emotion/miniXception_curves.png')
fig.show()

In [ ]:
from glob import glob
file_lst = sorted(glob('../models/weights/emotion/emotion_mini_XCEPTION*.hdf5'))
best_acc=0
best_idx=0
for i,filename in enumerate(file_lst):
    plot_conf_mat(model, load=True, load_loc=filename, 
              class_lst = 'Angry Disgust Fear Happy Sad Surprise Neutral', 
              out='model_weights/finalconf.png')
    if int(filename.split('.')[-2])>best_acc:
        best_acc = int(filename.split('.')[-2])
        best_idx=i
best_loc = file_lst[best_idx]
best_loc

In [ ]:
fig = plot_conf_mat(model, load=True, load_loc=best_loc, 
              class_lst = 'Angry Disgust Fear Happy Sad Surprise Neutral', 
              out='../models/emotion_conf.png')
fig.show()

In [ ]:
#model.summary()